## Interview times
**Goal:** Confirm that the time for each process in the survey workflow is measured correctly

### Imports

In [ ]:
from datetime import datetime, date
import sys
import os

sys.path.insert(0,os.path.abspath('..'))

import pandas as pd

import utils
plotly, Box, Histogram, Layout, Figure, Bar, Scatter, Marker = utils.setPlotly()

### Get the data

In [ ]:
allowedStatus = utils.allowedStatus
allowedDomains = utils.allowedDomains['greenville']

In [ ]:
ia = utils.getInterviewArchive()

In [ ]:
april4 = datetime(2016, 4, 4)
allFromApril4 = pd.DataFrame(list(ia.find({"createTime": {"$gte": april4}, "domainName": {"$in": allowedDomains}})))

### Grouped dataframes

Useful for the following visualizations

In [ ]:
allFromApril4["createDate"] = allFromApril4["createTime"].dt.date


completed = allFromApril4[(allFromApril4['status'] == 'completed') & (allFromApril4["domainName"].isin(allowedDomains))]
completed.loc[:, 'interviewTime'] = pd.to_timedelta(completed['finishTime'] - completed['createTime']).dt.seconds/60
completed.loc[:, 'ptIdleTime'] = pd.to_timedelta(completed['signTime'] - completed['finishTime']).dt.seconds/60
completed.loc[:, 'ptCareDeliveryTime'] = pd.to_timedelta(completed['signTime'] - completed['viewTime']).dt.seconds/60

completedGroupedMean = completed.groupby(['createDate', 'chiefComplaint'])
completedGroupedMean = completedGroupedMean.mean()
completedGroupedMean = completedGroupedMean.reset_index()

completedGroupedMedian = completed.groupby(['createDate', 'chiefComplaint'])
completedGroupedMedian = completedGroupedMedian.median()
completedGroupedMedian = completedGroupedMedian.reset_index()

In [ ]:
def plotHistogram(data, **kw):
    """
    Plot histogram with nice layout
    """
    layout = Layout(
        title=kw.get('title', ''),
        xaxis=dict(
            title=kw.get('xlabel', '')
        ),
        yaxis=dict(
            title=kw.get('ylabel', '')
        ),
        bargap=0.1,
        bargroupgap=0
    )
    bins = dict(start=0, end=100, size=1)
    plotly.offline.iplot(Figure(data=[Histogram(x=data, xbins=bins, autobinx=False)], layout=layout))

modules = pd.Series(completed["chiefComplaint"]).unique()
def plotStackBar(data, fields, title):
    """
    Plot stack bard with multiple fields
    """
    charts = []
    y = data
    for f in fields:
        charts.append(Bar(y=y[f], x=y['createDate'], name=f))

    layout = Layout(barmode='stack', title=title)
    figTimes = Figure(data=charts, layout=layout)
    plotly.offline.iplot(figTimes)

In [ ]:
plotHistogram(completed['interviewTime'], title="Histogram of Pt interview time", xlabel="Minutes", ylabel="Frequency")

In [ ]:
plotHistogram(completed['ptIdleTime'], title="Histogram of Pt idle time", xlabel="Minutes", ylabel="Frequency")

In [ ]:
plotly.offline.iplot(Figure(data=[Box(y=completed['ptIdleTime'])], layout=Layout( yaxis=dict(range=[0, 40]))))

In [ ]:
plotHistogram(completed['ptCareDeliveryTime'], title="Histogram of Pt care delivery time", xlabel="Minutes", ylabel="Frequency")